In [108]:
pip install xgboost

     ---------------------------------------- 89.1/89.1 MB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pylab as pltt

import statsmodels.api as sm

from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


In [2]:
train = pd.read_csv('train.csv')
train
testt = pd.read_csv('test.csv')

In [3]:
train


,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [4]:
train['color'].value_counts()

G    44391
E    35869
F    34258
H    30799
D    24286
I    17514
J     6456
Name: color, dtype: int64

In [5]:
tr_d = pd.get_dummies(data = train,drop_first=True)
tr_d.info()
tr_d['vol'] = tr_d['x']*tr_d['y']*tr_d['z']
# tr_d['depthvol'] = tr_d['vol']/tr_d['depth']
# tr_d.loc[tr_d["vol"] == 0, "depthvol"] = 0

trf = tr_d.drop(columns=['id'])
y = trf['price']
x = trf.drop(columns='price')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             193573 non-null  int64  
 1   carat          193573 non-null  float64
 2   depth          193573 non-null  float64
 3   table          193573 non-null  float64
 4   x              193573 non-null  float64
 5   y              193573 non-null  float64
 6   z              193573 non-null  float64
 7   price          193573 non-null  int64  
 8   cut_Good       193573 non-null  uint8  
 9   cut_Ideal      193573 non-null  uint8  
 10  cut_Premium    193573 non-null  uint8  
 11  cut_Very Good  193573 non-null  uint8  
 12  color_E        193573 non-null  uint8  
 13  color_F        193573 non-null  uint8  
 14  color_G        193573 non-null  uint8  
 15  color_H        193573 non-null  uint8  
 16  color_I        193573 non-null  uint8  
 17  color_J        193573 non-nul

In [6]:

data_train, data_test, y_train, y_test = tts(x,y,train_size = 0.8, test_size = 0.2, random_state = 100)


In [7]:
data_train_sm = sm.add_constant(data_train)
data_train_sm.corr()

,const,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,vol
const,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carat,NaN,1.000000,0.028798,0.225844,0.980609,0.980204,0.973626,0.047471,-0.190657,0.141955,...,0.174507,0.154091,-0.111576,0.084434,0.262142,-0.071996,-0.031269,-0.174918,-0.143146,0.998665
depth,NaN,0.028798,1.000000,-0.229655,-0.009756,-0.010716,0.082641,0.233243,-0.054695,-0.210368,...,0.023029,0.017438,-0.023658,0.050076,0.021703,-0.027110,-0.017544,-0.022930,-0.018855,0.016146
table,NaN,0.225844,-0.229655,1.000000,0.237747,0.233431,0.212335,0.154614,-0.624331,0.454570,...,0.019622,0.040256,-0.082896,0.066207,0.115728,-0.050431,-0.012060,-0.078678,-0.070484,0.217776
x,NaN,0.980609,-0.009756,0.237747,1.000000,0.999083,0.988969,0.045489,-0.192101,0.151404,...,0.155097,0.137222,-0.125837,0.102846,0.265913,-0.070153,-0.031049,-0.195823,-0.156415,0.980810
y,NaN,0.980204,-0.010716,0.233431,0.999083,1.000000,0.988734,0.046915,-0.188032,0.139995,...,0.155421,0.137428,-0.125071,0.102988,0.264594,-0.069703,-0.031436,-0.195390,-0.155448,0.980465
z,NaN,0.973626,0.082641,0.212335,0.988969,0.988734,1.000000,0.066786,-0.193003,0.125170,...,0.155685,0.137579,-0.126096,0.106155,0.264110,-0.071808,-0.031859,-0.195691,-0.156219,0.976253
cut_Good,NaN,0.047471,0.233243,0.154614,0.045489,0.046915,0.066786,1.000000,-0.241445,-0.148745,...,0.005643,0.017003,-0.029073,0.057214,0.049569,-0.028081,-0.025862,-0.036713,-0.037634,0.040876
cut_Ideal,NaN,-0.190657,-0.054695,-0.624331,-0.192101,-0.188032,-0.193003,-0.241445,1.000000,-0.563389,...,-0.012740,-0.040307,0.091519,-0.095472,-0.114795,0.053051,0.026705,0.087406,0.078231,-0.181645
cut_Premium,NaN,0.141955,-0.210368,0.454570,0.151404,0.139995,0.125170,-0.148745,-0.563389,1.000000,...,0.001519,0.015070,-0.049700,0.028161,0.072938,-0.024146,0.014703,-0.056705,-0.062412,0.140351


In [8]:
data_train = data_train.dropna()
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154858 entries, 126278 to 38408
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   carat          154858 non-null  float64
 1   depth          154858 non-null  float64
 2   table          154858 non-null  float64
 3   x              154858 non-null  float64
 4   y              154858 non-null  float64
 5   z              154858 non-null  float64
 6   cut_Good       154858 non-null  uint8  
 7   cut_Ideal      154858 non-null  uint8  
 8   cut_Premium    154858 non-null  uint8  
 9   cut_Very Good  154858 non-null  uint8  
 10  color_E        154858 non-null  uint8  
 11  color_F        154858 non-null  uint8  
 12  color_G        154858 non-null  uint8  
 13  color_H        154858 non-null  uint8  
 14  color_I        154858 non-null  uint8  
 15  color_J        154858 non-null  uint8  
 16  clarity_IF     154858 non-null  uint8  
 17  clarity_SI1    154858 non

In [97]:
# data_train.drop(columns = ['x', 'y', 'z'])

,carat,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,vol
126278,0.70,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,114.841233
28276,0.58,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,93.314700
17396,0.39,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,63.850644
15721,0.39,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,62.132994
2204,2.02,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,333.107775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82270,0.35,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,56.901376
65615,0.70,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,113.786913
77655,1.23,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,200.065726
56088,0.30,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,49.228560


In [9]:
data_trainv = data_train.values


In [10]:
model=XGBRegressor(max_depth=10, learning_rate=0.24 , n_estimators=50, objective='reg:linear', booster='gbtree')
XGB=model.fit(data_trainv,y_train)


[17:05:18] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [11]:
data_testv = data_test.values
prediction=XGB.predict(data_testv)

In [12]:
test_pred = XGB.predict(data_testv)
test_pred

array([ 6017.79  ,  9553.303 ,  4244.0737, ...,  9390.86  , 15039.044 ,
        1734.2535], dtype=float32)

In [98]:
lr = sm.OLS(y_train, data_train).fit()
lr.params

carat            5679.944376
x                -925.163536
y                -246.900828
z               -1399.016836
cut_Good          682.085051
cut_Ideal         862.410067
cut_Premium       761.717546
cut_Very Good     772.890825
color_E          -154.670811
color_F          -233.563271
color_G          -349.008864
color_H          -849.401565
color_I         -1393.421274
color_J         -2295.243449
clarity_IF       5273.688390
clarity_SI1      3931.257980
clarity_SI2      3014.969779
clarity_VS1      4807.510399
clarity_VS2      4535.736334
clarity_VVS1     5082.466783
clarity_VVS2     5084.029706
vol                50.904992
dtype: float64

In [14]:
# y_train_pred = lr.predict(data_train)
res = (y_test - test_pred)
res

3033       807.209961
136642    -839.302734
190247     293.926270
20099     -100.353149
90239      -45.358154
             ...     
133025   -1394.635742
182406      -5.671143
96657     -360.860352
166581     421.956055
63967      -40.253540
Name: price, Length: 38715, dtype: float32

In [100]:

y_test_pred = lr.predict(data_test)
y_test_pred

3033       6146.781879
136642     9766.492270
190247     4803.552760
20099      1093.206559
90239       874.346001
              ...     
133025     9642.619123
182406     1449.283352
96657      7433.829696
166581    11622.385709
63967      2123.450192
Length: 38715, dtype: float64

In [15]:
from sklearn.metrics import r2_score

# Checking the R-squared value
r_squared = r2_score(y_test, test_pred)
r_squared

0.9780838609185085

In [16]:
# testt_sm = sm.add_constant(testt)
test = pd.get_dummies(data = testt,drop_first=True)
test =test.drop(columns=['id'])
test['vol'] = test['x']*test['y']*test['z']
# test.drop(columns = ['x', 'y', 'z'])

In [17]:
test

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,vol
0,0.35,62.3,56.0,4.51,4.54,2.82,0,1,0,0,...,0,0,0,0,0,0,1,0,0,57.740628
1,0.77,62.8,56.0,5.83,5.87,3.68,0,0,0,1,...,0,0,0,0,1,0,0,0,0,125.937328
2,0.71,61.9,53.0,5.77,5.74,3.55,0,1,0,0,...,1,0,0,0,0,0,1,0,0,117.575290
3,0.33,61.6,55.0,4.44,4.42,2.73,0,1,0,0,...,0,0,0,0,0,0,0,0,1,53.575704
4,1.20,62.7,56.0,6.75,6.79,4.24,0,0,0,1,...,1,0,0,0,0,0,1,0,0,194.329800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129045,0.72,62.0,56.0,5.75,5.78,3.57,0,1,0,0,...,0,0,0,0,0,0,0,0,1,118.648950
129046,0.70,59.6,62.0,5.77,5.74,3.43,0,0,1,0,...,0,0,0,1,0,0,0,0,0,113.600914
129047,1.01,62.3,58.0,6.44,6.41,4.01,0,0,1,0,...,0,0,0,0,0,0,0,0,1,165.534404
129048,1.35,62.0,56.0,7.05,7.08,4.38,0,1,0,0,...,0,0,0,0,0,0,0,0,0,218.623320


In [18]:
y_test_final=XGB.predict(test)
# y_test_final = lr.predict(test)
y_test_final = pd.DataFrame(y_test_final)



fdf = pd.DataFrame(columns = ['id', 'price'])
fdf['id'] = testt['id']
fdf['price'] = y_test_final[0]


fdf = fdf.set_index('id')
fdf

,price
id,
193573,863.129822
193574,2517.803711
193575,2356.838867
193576,849.392090
193577,5875.446289
...,...
322618,3649.322754
322619,2586.278320
322620,7663.449219


In [19]:
fdf.to_csv('submission.csv')